In [ ]:
from whar_datasets.adapters.torch_adapter import TorchAdapter
from whar_datasets.support.getter import WHARDatasetID, get_whar_cfg

In [ ]:
# initialize config and dataset given a dataset id
cfg = get_whar_cfg(dataset_id=WHARDatasetID.WISDM)
dataset = TorchAdapter(cfg=cfg)

# preprocess and postprocess for a given fold
dataset.preprocess()
dataset.postprocess(fold_index=0)

# get dataloaders for a given batch size
dataloaders = dataset.get_dataloaders(batch_size=32)